In [1]:
import pandas as pd
import random
import json
import re

import google.generativeai as genai
from openai import OpenAI
from dotenv import load_dotenv
import os

from tqdm import tqdm

In [2]:
load_dotenv()
openai_key = os.getenv("OPENAI_API_KEY")

In [3]:
import openai

def prompt_model(system_prompt, message, sample_prompt=None, sample_answer=None, model_name='gpt-4'):
    client = OpenAI(api_key=openai_key)
    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role":"system", "content": system_prompt},
            {"role": "user", "content": message}
        ],
        response_format={
            "type":"json_object"
        },
        temperature=0.6
    )
    
    return completion.choices[0].message.content

def clean_sentences(response):
    json_content = response

    json_str = json_content[json_content.index('['):len(json_content) - json_content[::-1].index(']')]
    cleaned_json_str = re.sub(r',\s*([\]}])', r'\1', json_str)

    try:
        json_obj = json.loads(cleaned_json_str)
    except json.JSONDecodeError as e:
        json_obj = json.loads(cleaned_json_str) 

    return json_obj


In [4]:

system_prompt = 'You are a helpful chat assistant ready to carry out the user\'s requests.'
user_prompt = 'You are helping me to generate a dataset for training very very small language models.' + \
             'Please can you generate 50 sentences that are no more than 10 words each, with very very simple grammar and using only VERY SIMPLE words - restrict to the 500 most common english words.' + \
             'The theme of the sentences is: {}. I\'d like the output to be in the format of a JSON list of strings.'

In [5]:
themes = [
    "children", "weather", "fun", "animals", "food", "sports", "friendship", "family", "nature", "colors",
    "hobbies", "travel", "music", "love", "work", "technology", "dreams", "learning", "health", "adventure",
    "art", "school", "emotions", "seasons", "flowers", "computers", "ocean", "festivals", "books", "movies",
    "holidays", "games", "clothes", "time", "plants", "cities", "mountains", "cars", "shopping", "memories",
    "houses", "neighbors", "language", "history", "insects", "wildlife", "gardens", "streets", "festivities", "stories"
]


In [9]:
for theme in tqdm(themes[27:]):
    for i in range(5):
        response = prompt_model(system_prompt, user_prompt.format(theme))
        sentences = clean_sentences(response)
        with open('sentences.txt', 'a') as f:
            for sentence in sentences:
                f.write(sentence + '\n')


100%|██████████| 23/23 [16:09<00:00, 42.15s/it]


In [ ]:
sentences = clean_sentences(response)

In [ ]:
with open('sentences.txt', 'a') as f:
    for sentence in sentences:
        f.write(sentence + '\n')